In [371]:
import pandas as pd
import altair as alt
import arrow

import src.logic.severa.base_client
import src.logic.severa.fetch
import src.logic.severa.client
import src.logic.severa.models as models
import src.util.process
import src.database.database
from src.util.daterange import DateRange
import src.util.stable_hash
import datetime

import src.logic.processing
import src.logic.processing_pandera

reload(src.logic.processing_pandera)

reload(src.logic.severa.base_client)
reload(src.logic.severa.fetch)
reload(src.logic.severa.client)
reload(src.logic.processing)
reload(src.util.process)
reload(src.database.database)
reload(src.util.daterange)
reload(src.util.stable_hash)

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [271]:
b = src.database.database.Base("kpi-dev-02", "billing")
billings_all = b.find({})
# billings_processed = ProcessBilling(billing_p.result()).process(
#             span.start.datetime, span.end.datetime
#         )

2023-07-15 15:46:33.661 | INFO     | src.database.database:find:91 - [billing] Query '{}' resulted in 2126 results in 3.70s.


In [372]:
history = await src.logic.processing.load_merge_billing_forecast_history()
history

2023-07-15 23:40:28.370 | INFO     | src.database.database:find:91 - [billing] Query '{}' resulted in 2126 results in 2.39s.
2023-07-15 23:40:34.723 | SUCCESS  | src.logic.severa.base_client:get_with_retries:196 - HTTP/2 GET users in 0.22s.


,id,project,value,user,forecast_date,date,forecast_length,first_name,last_name,business_unit
0,billing,bbe91a59-1ae7-0230-0aea-090c8f31817e,4.761905,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-21 00:00:00+00:00,0,Lari,Ruotsalainen,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
1,billing,bbe91a59-1ae7-0230-0aea-090c8f31817e,4.761905,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-22 00:00:00+00:00,1,Lari,Ruotsalainen,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
2,billing,bbe91a59-1ae7-0230-0aea-090c8f31817e,0.000000,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-23 00:00:00+00:00,2,Lari,Ruotsalainen,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
3,billing,bbe91a59-1ae7-0230-0aea-090c8f31817e,0.000000,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-24 00:00:00+00:00,3,Lari,Ruotsalainen,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
4,billing,bbe91a59-1ae7-0230-0aea-090c8f31817e,0.000000,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-25 00:00:00+00:00,4,Lari,Ruotsalainen,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
...,...,...,...,...,...,...,...,...,...,...
57291,billing,636b2177-27c2-348f-9840-36c8ff3a5aa9,41.666667,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-27 00:00:00+00:00,12,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
57292,billing,636b2177-27c2-348f-9840-36c8ff3a5aa9,41.666667,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-28 00:00:00+00:00,13,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
57293,billing,636b2177-27c2-348f-9840-36c8ff3a5aa9,0.000000,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-29 00:00:00+00:00,14,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e
57294,billing,636b2177-27c2-348f-9840-36c8ff3a5aa9,0.000000,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-30 00:00:00+00:00,15,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e


In [381]:
h = history.copy()
h = h[h.forecast_length <= 7]
g = h.groupby(["forecast_date", "first_name"])
d = pd.concat([g.value.sum()], axis=1).reset_index()
d[d.first_name == "Ville"]
# g2 = d.groupby(["date", "first_name"])
# d2 = pd.concat([g.value.sum(), g.date.max()], axis=1).reset_index()

,forecast_date,first_name,value
6,2023-06-21 00:00:00+00:00,Ville,1023.809524
14,2023-06-22 00:00:00+00:00,Ville,1254.761905
22,2023-06-23 00:00:00+00:00,Ville,1285.833333
31,2023-06-24 00:00:00+00:00,Ville,1285.833333
40,2023-06-25 00:00:00+00:00,Ville,1285.833333
49,2023-06-26 00:00:00+00:00,Ville,1285.833333
58,2023-06-27 00:00:00+00:00,Ville,1028.666667
67,2023-06-28 00:00:00+00:00,Ville,914.357143
76,2023-06-29 00:00:00+00:00,Ville,838.095238
79,2023-06-30 00:00:00+00:00,Ville,428.571429


In [350]:
b0 = billings_all.copy()

b0.start_date = pd.to_datetime(b0.start_date, utc=True)
b0.end_date = pd.to_datetime(b0.end_date, utc=True)
b0.forecast_date = pd.to_datetime(b0.forecast_date, utc=True)

d = arrow.utcnow()
start = d.shift(days=-14).floor("day")
end = d.ceil("day")

# find_most_recent_forecast_for_span
latest_date = b0[b0.forecast_date <= start.date().isoformat()].forecast_date.max()
latest_forecasts = b0[b0.forecast_date == latest_date]
latest_forecasts.dtypes

# unravel
print(latest_forecasts.value.sum())
unrav = src.logic.processing.ProcessBilling(latest_forecasts.copy()).process(
    latest_date.to_pydatetime(), end.datetime
)
unrav.unraveled

100686.6


,id,internal_guid,start_date,end_date,project,value,billing,expense,revenue,labor_expense,user,forecast_date,date
14,billing,8ab80be6-9735-0690-85be-43e60f30be08,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,1ad42d62-f0ac-1445-32e9-05b8868ef98f,0.0,800.0,0.0,0.0,0.0,2f365dcd-6ada-f6d8-8d73-4f393745e513,2023-07-01 00:00:00+00:00,2023-07-15 00:00:00+00:00
997,billing,e57e351e-4e88-382e-244e-fd1977b3c7a3,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,af008ee1-a17d-8268-63ae-cd1e898519f9,0.0,1000.0,0.0,0.0,0.0,2f365dcd-6ada-f6d8-8d73-4f393745e513,2023-07-01 00:00:00+00:00,2023-07-15 00:00:00+00:00
1640,billing,aa911283-7be7-81ed-52b5-b0e07b505643,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,d9dafcdb-b1f6-8af8-eb7d-861da9373a0f,0.0,200.0,0.0,0.0,0.0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-01 00:00:00+00:00,2023-07-15 00:00:00+00:00
2070,billing,8bea52d0-9be0-2688-f492-7d943bf3dcfb,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,82f8a4df-ea8a-dd8b-2a3a-366296048e3d,0.0,1000.0,0.0,0.0,0.0,2ed8156d-f3f9-2212-c739-b0d9f7ac81ca,2023-07-01 00:00:00+00:00,2023-07-15 00:00:00+00:00


In [361]:
import src.logic.processing_pandera

reload(src.logic.processing_pandera)

b0 = billings_all.copy()

p = src.logic.processing_pandera

spaan = DateRange(start, end)

p.process_billing_forecasts(b0)

# unr = p.unravel(p.expand_start_and_end(b0, spaan))
# unr = p.calculate_weekday_statistics(unr)
# unr = p.recalculate_values(unr, p.ValueRecalculationMethod.SCALE_VALUE_TO_NUMBER_OF_WORKDAYS, True)
# # unr = p.cull(unr, spaan)
# unr

# # Pitää valita viimeisimpien ennusteiden joukosta
# # ne, joiden date on oikeassa spanissa

# # groupby(forecast_date)
# # x = date - forecast_date (esim. 4)
# # nyt halutaan valita (groupittain) kaikki, joiden x <= len(span)
# # eli jotka ovat x tulevan päivän ennustetta
# # n vrk ennuste päivänä date = filter(x < n).groupby(forecast_date).sum()

# n = 2
# import operator
# unr["forecast_length"] = unr["date"] - unr["forecast_date"]
# print("!!!", unr.forecast_length.map(operator.attrgetter("days")).min())
# unr = unr[unr.forecast_length.map(operator.attrgetter("days")) >= 0]
# unr2 = unr[unr.forecast_length.map(operator.attrgetter("days")) < n]
# grps = unr2.groupby("forecast_date")
# pd.concat([grps["date"].max(), grps["value"].sum(), grps["value"].count()], axis=1)
# unr2[unr2.forecast_date == "2023-06-22"]
# # unr3 = unr2.groupby("forecast_date")["value"].sum().reset_index()
# # unr3["forecasted_date"] = unr3["forecast_date"] + pd.Timedelta(days=len(spaan))
# # unr3

id                    string[python]
internal_guid         string[python]
start_date       datetime64[ns, UTC]
end_date         datetime64[ns, UTC]
project               string[python]
value                        float64
billing                      float64
expense                      float64
revenue                      float64
labor_expense                float64
user                  string[python]
forecast_date    datetime64[ns, UTC]
dtype: object
        id                         internal_guid                start_date  \
0  billing  568feff7-bf6a-45bd-702e-d0378756b2a9 2023-06-01 00:00:00+00:00   
1  billing  2e1893e7-2a19-f04a-84eb-77f36d1e9b66 2023-08-01 00:00:00+00:00   
2  billing  cca04799-ff11-d23f-4b22-ec85d68997cf 2023-09-01 00:00:00+00:00   

                          end_date                               project  \
0 2023-06-30 23:59:59.999000+00:00  bbe91a59-1ae7-0230-0aea-090c8f31817e   
1 2023-08-31 23:59:59.999000+00:00  bbe91a59-1ae7-0230-0aea-090c8f31817e   
2

,id,internal_guid,start_date,end_date,project,value,billing,expense,revenue,labor_expense,user,forecast_date,_date,date,_is_workday,_num_all_days,_num_work_days,forecast_length
20,billing,568feff7-bf6a-45bd-702e-d0378756b2a9,2023-06-01 00:00:00+00:00,2023-06-30 23:59:59.999000+00:00,bbe91a59-1ae7-0230-0aea-090c8f31817e,4.761905,100.0,0.0,0.0,0.0,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-21 00:00:00+00:00,2023-06-21 00:00:00+00:00,True,29,21,0
21,billing,568feff7-bf6a-45bd-702e-d0378756b2a9,2023-06-01 00:00:00+00:00,2023-06-30 23:59:59.999000+00:00,bbe91a59-1ae7-0230-0aea-090c8f31817e,4.761905,100.0,0.0,0.0,0.0,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-22 00:00:00+00:00,2023-06-22 00:00:00+00:00,True,29,21,1
22,billing,568feff7-bf6a-45bd-702e-d0378756b2a9,2023-06-01 00:00:00+00:00,2023-06-30 23:59:59.999000+00:00,bbe91a59-1ae7-0230-0aea-090c8f31817e,0.000000,100.0,0.0,0.0,0.0,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-23 00:00:00+00:00,2023-06-23 00:00:00+00:00,False,29,21,2
23,billing,568feff7-bf6a-45bd-702e-d0378756b2a9,2023-06-01 00:00:00+00:00,2023-06-30 23:59:59.999000+00:00,bbe91a59-1ae7-0230-0aea-090c8f31817e,0.000000,100.0,0.0,0.0,0.0,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-24 00:00:00+00:00,2023-06-24 00:00:00+00:00,False,29,21,3
24,billing,568feff7-bf6a-45bd-702e-d0378756b2a9,2023-06-01 00:00:00+00:00,2023-06-30 23:59:59.999000+00:00,bbe91a59-1ae7-0230-0aea-090c8f31817e,0.000000,100.0,0.0,0.0,0.0,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-21 00:00:00+00:00,2023-06-25 00:00:00+00:00,2023-06-25 00:00:00+00:00,False,29,21,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65023,billing,5b4ace44-4f0e-71da-a6df-edd4c46dc72a,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,636b2177-27c2-348f-9840-36c8ff3a5aa9,41.666667,875.0,0.0,0.0,0.0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-27 00:00:00+00:00,2023-07-27 00:00:00+00:00,True,30,21,12
65024,billing,5b4ace44-4f0e-71da-a6df-edd4c46dc72a,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,636b2177-27c2-348f-9840-36c8ff3a5aa9,41.666667,875.0,0.0,0.0,0.0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-28 00:00:00+00:00,2023-07-28 00:00:00+00:00,True,30,21,13
65025,billing,5b4ace44-4f0e-71da-a6df-edd4c46dc72a,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,636b2177-27c2-348f-9840-36c8ff3a5aa9,0.000000,875.0,0.0,0.0,0.0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-29 00:00:00+00:00,2023-07-29 00:00:00+00:00,False,30,21,14
65026,billing,5b4ace44-4f0e-71da-a6df-edd4c46dc72a,2023-07-01 00:00:00+00:00,2023-07-31 23:59:59.999000+00:00,636b2177-27c2-348f-9840-36c8ff3a5aa9,0.000000,875.0,0.0,0.0,0.0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-15 00:00:00+00:00,2023-07-30 00:00:00+00:00,2023-07-30 00:00:00+00:00,False,30,21,15


In [210]:
span = DateRange(arrow.get("2023-07-13").shift(days=-60), arrow.get("2023-07-13"))
s = await src.logic.processing.load_and_merge(span)

2023-07-13 22:20:35.461 | DEBUG    | src.logic.processing:load_and_merge:528 - span_severa=<DateRange [2023-05-14 .. 2023-07-13]>
2023-07-13 22:20:35.462 | DEBUG    | src.logic.processing:load_and_merge:530 - span_future=<DateRange [Empty]>
2023-07-13 22:20:35.462 | DEBUG    | src.logic.processing:load_and_merge:532 - forecasts_from_database=False
2023-07-13 22:20:35.470 | DEBUG    | src.logic.processing:load_and_merge:536 - Creating fetch tasks.
2023-07-13 22:20:35.471 | DEBUG    | src.logic.processing:load_and_merge:545 - Fetching from db.
2023-07-13 22:20:35.472 | DEBUG    | src.logic.processing:load_and_merge:557 - Fetching from db done.
2023-07-13 22:20:35.877 | SUCCESS  | src.logic.severa.base_client:get_with_retries:192 - HTTP/2 GET users in 0.16s.
2023-07-13 22:20:35.904 | SUCCESS  | src.logic.severa.base_client:get_with_retries:192 - HTTP/2 GET users in 0.16s.
2023-07-13 22:20:35.937 | SUCCESS  | src.logic.severa.base_client:get_with_retries:192 - HTTP/2 GET users in 0.17s.
20

In [253]:
ss = s.copy()

prod_work = s[(s.id == "workhours") & s.productive].copy()
prod_work["id"] = "workhours_productive"
unprod_work = s[(s.id == "workhours") & ~s.productive].copy()
unprod_work["id"] = "workhours_unproductive"

sss = pd.concat([ss, prod_work, unprod_work], ignore_index=True)

piv = sss.pivot_table(
    values=["value"],
    index=["date", "first_name"],
    columns=["id"],
    aggfunc="sum",
    fill_value=0,
)
piv.columns = piv.columns.droplevel(0)
piv["total_hours"] = piv["absences"] + piv["workhours"]
piv["cost"] = piv["total_hours"] * piv["hour_cost"]
piv["margin"] = piv["billing"] - piv["cost"]
windowed = piv.groupby(["date"]).sum().rolling(window=7).sum()
windowed["margin%"] = windowed["margin"] / windowed["billing"]
windowed["laskutusaste"] = windowed["workhours_productive"] / windowed["workhours"]
windowed.reset_index().iloc[[-7, -1]].diff().iloc[-1]

id
date                      6 days 00:00:00
absences                             25.0
billing                           -1125.0
hour_cost                             0.0
maximum                               0.0
salesvalue                        -7075.0
workhours                           -66.5
workhours_productive                -34.0
workhours_unproductive              -32.5
total_hours                         -41.5
cost                              -1989.0
margin                              864.0
margin%                         -5.382964
laskutusaste                     0.162596
Name: 60, dtype: object

In [226]:
piv

id                                                              absences  \
date                      user                                             
2023-05-14 00:00:00+00:00 0d100bc1-8376-7ac5-7bbb-055cdc20497d      0.00   
                          2441bfbf-78f2-0173-b0a1-8634ce167a0d      0.00   
                          2f365dcd-6ada-f6d8-8d73-4f393745e513      0.00   
                          30c9d126-7cdc-53fc-3b05-7f98b25e3c98      0.00   
                          5cd93092-3385-3e89-003a-9fe6d03a424d      0.00   
...                                                                  ...   
2023-07-13 00:00:00+00:00 5cd93092-3385-3e89-003a-9fe6d03a424d      0.00   
                          78ddf0fe-9e39-5e5e-5096-a80409925de9      7.50   
                          a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8      6.25   
                          af48336f-0979-b971-4927-9b09d53a3cef      0.00   
                          eb8d73da-8c1c-7996-b085-332e777dff1a      6.00   

id                                                              billing  \
date                      user                                            
2023-05-14 00:00:00+00:00 0d100bc1-8376-7ac5-7bbb-055cdc20497d      0.0   
                          2441bfbf-78f2-0173-b0a1-8634ce167a0d      0.0   
                          2f365dcd-6ada-f6d8-8d73-4f393745e513      0.0   
                          30c9d126-7cdc-53fc-3b05-7f98b25e3c98      0.0   
                          5cd93092-3385-3e89-003a-9fe6d03a424d      0.0   
...                                                                 ...   
2023-07-13 00:00:00+00:00 5cd93092-3385-3e89-003a-9fe6d03a424d      0.0   
                          78ddf0fe-9e39-5e5e-5096-a80409925de9      0.0   
                          a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8      0.0   
                          af48336f-0979-b971-4927-9b09d53a3cef      0.0   
                          eb8d73da-8c1c-7996-b085-332e777dff1a      0.0   

id                                                              hour_cost  \
date                      user                                              
2023-05-14 00:00:00+00:00 0d100bc1-8376-7ac5-7bbb-055cdc20497d       50.0   
                          2441bfbf-78f2-0173-b0a1-8634ce167a0d       44.0   
                          2f365dcd-6ada-f6d8-8d73-4f393745e513       50.0   
                          30c9d126-7cdc-53fc-3b05-7f98b25e3c98       48.5   
                          5cd93092-3385-3e89-003a-9fe6d03a424d       95.0   
...                                                                   ...   
2023-07-13 00:00:00+00:00 5cd93092-3385-3e89-003a-9fe6d03a424d       95.0   
                          78ddf0fe-9e39-5e5e-5096-a80409925de9       52.0   
                          a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8       43.0   
                          af48336f-0979-b971-4927-9b09d53a3cef       56.0   
                          eb8d73da-8c1c-7996-b085-332e777dff1a       48.5   

id                                                              maximum  \
date                      user                                            
2023-05-14 00:00:00+00:00 0d100bc1-8376-7ac5-7bbb-055cdc20497d      0.0   
                          2441bfbf-78f2-0173-b0a1-8634ce167a0d      0.0   
                          2f365dcd-6ada-f6d8-8d73-4f393745e513      0.0   
                          30c9d126-7cdc-53fc-3b05-7f98b25e3c98      0.0   
                          5cd93092-3385-3e89-003a-9fe6d03a424d      0.0   
...                                                                 ...   
2023-07-13 00:00:00+00:00 5cd93092-3385-3e89-003a-9fe6d03a424d      1.5   
                          78ddf0fe-9e39-5e5e-5096-a80409925de9      7.5   
                          a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8      7.5   
                          af48336f-0979-b971-4927-9b09d53a3cef      7.5   
                          eb8d73da-8c1c-7996-b085-332e777dff1a      7.5   

id                                                       

In [205]:
ss = s.copy()
ss.productive = ss.productive.fillna(True)
# ss.pivot_table(values=["value"], index=[pd.Grouper(key="date", freq="W")], columns=["id", "productive"], margins=True, observed=False, dropna=False, aggfunc="sum", fill_value=0)
ss.groupby([pd.Grouper(key="date", freq="W"), "id", "productive", "first_name"])[
    "value"
].sum().unstack()
# ss[ss.id == "salesvalue"]

first_name                                         Johanna       Joni  Lari  \
date                      id          productive                              
2023-07-02 00:00:00+00:00 absences    False       0.000000   0.000000   0.0   
                                      True        0.000000   0.000000   0.0   
                          billing     False       0.000000   0.000000   0.0   
                                      True        0.000000   0.000000   0.0   
                          daily_hours False       0.000000   0.000000   0.0   
...                                                    ...        ...   ...   
2023-07-30 00:00:00+00:00 salesvalue  True        0.000000   0.000000   0.0   
                          saleswork   False       0.000000   0.000000   0.0   
                                      True        0.663871   2.625000   0.0   
                          workhours   False       0.000000  18.639053   0.0   
                                      True        1.228933  10.318170   0.0   

first_name                                        Margarita     Miika  Panu  \
date                      id          productive                              
2023-07-02 00:00:00+00:00 absences    False        0.000000  0.000000   0.0   
                                      True         0.000000  0.000000   0.0   
                          billing     False        0.000000  0.000000   0.0   
                                      True         0.000000  0.000000   0.0   
                          daily_hours False        0.000000  0.000000   0.0   
...                                                     ...       ...   ...   
2023-07-30 00:00:00+00:00 salesvalue  True         0.000000  0.000000   0.0   
                          saleswork   False        0.000000  0.000000   0.0   
                                      True         0.000000  0.000000   0.0   
                          workhours   False        0.000000  2.509960   0.0   
                                      True        16.463181  0.887868   0.0   

first_name                                            Topi  Ville  Vladimirs  
date                      id          productive                              
2023-07-02 00:00:00+00:00 absences    False       0.000000    0.0    0.00000  
                                      True        0.000000    0.0    0.00000  
                          billing     False       0.000000    0.0    0.00000  
                                      True        0.000000    0.0    0.00000  
                          daily_hours False       0.000000    0.0    0.00000  
...                                                    ...    ...        ...  
2023-07-30 00:00:00+00:00 salesvalue  True        0.000000    0.0    0.00000  
                          saleswork   False       0.000000    0.0    0.00000  
                                      True        0.000000    0.0    0.00000  
                          workhours   False       0.000000    0.0    5.01992  
                                      True        0.734314    0.0   22.63230  

[70 rows x 9 columns]

In [46]:
uu = src.logic.processing.ProcessUsers(u.copy())
uu.data

uu.process(span.start.datetime, span.end.datetime).unraveled

,user,first_name,last_name,business_unit,start_date,end_date,id,value,_id,_date,_num_days,_num_workdays,_pre_scale,_zero_if_holiday,_is_workday,date
212,eb8d73da-8c1c-7996-b085-332e777dff1a,Vladimirs,Davidko,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,daily_hours,7.5,AGWTIlrCShg7HMrbudrkQw==,2023-08-01 00:00:00+00:00,242.0,168.0,1.0,True,True,2023-08-01 00:00:00+00:00
213,eb8d73da-8c1c-7996-b085-332e777dff1a,Vladimirs,Davidko,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,daily_hours,7.5,AGWTIlrCShg7HMrbudrkQw==,2023-08-02 00:00:00+00:00,242.0,168.0,1.0,True,True,2023-08-02 00:00:00+00:00
214,eb8d73da-8c1c-7996-b085-332e777dff1a,Vladimirs,Davidko,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,daily_hours,7.5,AGWTIlrCShg7HMrbudrkQw==,2023-08-03 00:00:00+00:00,242.0,168.0,1.0,True,True,2023-08-03 00:00:00+00:00
215,eb8d73da-8c1c-7996-b085-332e777dff1a,Vladimirs,Davidko,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,daily_hours,7.5,AGWTIlrCShg7HMrbudrkQw==,2023-08-04 00:00:00+00:00,242.0,168.0,1.0,True,True,2023-08-04 00:00:00+00:00
216,eb8d73da-8c1c-7996-b085-332e777dff1a,Vladimirs,Davidko,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,daily_hours,0.0,AGWTIlrCShg7HMrbudrkQw==,2023-08-05 00:00:00+00:00,242.0,168.0,1.0,True,False,2023-08-05 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36337,2441bfbf-78f2-0173-b0a1-8634ce167a0d,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-02 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,hour_cost,44.0,ALEU+hmqdwoJbmHZTWdU3g==,2023-08-27 00:00:00+00:00,241.0,168.0,1.0,False,False,2023-08-27 00:00:00+00:00
36338,2441bfbf-78f2-0173-b0a1-8634ce167a0d,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-02 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,hour_cost,44.0,ALEU+hmqdwoJbmHZTWdU3g==,2023-08-28 00:00:00+00:00,241.0,168.0,1.0,False,True,2023-08-28 00:00:00+00:00
36339,2441bfbf-78f2-0173-b0a1-8634ce167a0d,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-02 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,hour_cost,44.0,ALEU+hmqdwoJbmHZTWdU3g==,2023-08-29 00:00:00+00:00,241.0,168.0,1.0,False,True,2023-08-29 00:00:00+00:00
36340,2441bfbf-78f2-0173-b0a1-8634ce167a0d,Margarita,Borisova,f6d9f1e8-afae-1a74-5bbd-54d840a3e40e,2023-01-02 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,hour_cost,44.0,ALEU+hmqdwoJbmHZTWdU3g==,2023-08-30 00:00:00+00:00,241.0,168.0,1.0,False,True,2023-08-30 00:00:00+00:00


In [17]:
span = DateRange(arrow.get("2023-08-01"), arrow.get("2023-08-31"))

all_projs = list(projs.values())
subset_projs = [p for p in all_projs if (not p.isClosed and not p.isInternal)]
# and p.lastUpdatedDateTime > span.start.shift(months=-4).datetime)]


print("len: ", len(all_projs), " / ", len(subset_projs))

myllymatka = "7dbc27c9-7b75-5b5b-3fb4-8d67689c6a8b"

print("Onko myllymatka all? ", myllymatka in [p.guid for p in all_projs])
print("Onko myllymatka subset? ", myllymatka in [p.guid for p in subset_projs])

m = next(p for p in all_projs if p.guid == myllymatka)
print(m.model_dump(mode="json"))

async with src.logic.severa.client.Client() as f:
    m_forecasts_all = await f._client.get_all(f"projects/{myllymatka}/projectforecasts")
    m_forecasts_span = await f._client.get_all(
        f"projects/{myllymatka}/projectforecasts", {**span}
    )

    print("len of forecasts: ", len(m_forecasts_all), " / ", len(m_forecasts_span))

len:  461  /  137
Onko myllymatka all?  True
Onko myllymatka subset?  True
{'projectStatus': {'name': 'Kunnossa', 'description': 'Vaihdettu asiakas → #25126, jota sovittu käytettävän ATT:n hankkeissa.', 'projectStatusTypeGuid': '5ec93fb3-69ac-e620-9d4b-03ead8054b85', 'guid': '782ffea0-f7c6-64e0-cd5f-ab73b83ea110'}, 'isInternal': False, 'internalName': 'Haso Myllymatka', 'number': 4027, 'description': 'Projektipankki SokoPro, Villen tai Topin tunnarit. Mallit Daluxissa, Villen tunnarit.', 'isClosed': False, 'closedDate': None, 'isJoiningAllowed': True, 'customer': {'guid': '0974ee14-c8f4-ee4a-5e1b-dff6c1326622', 'name': 'Helsingin kaupunki / KYMP / RYA / Asuntotuotanto', 'number': 25126}, 'projectOwner': {'code': '125', 'name': 'Ville Reima', 'guid': '2f365dcd-6ada-f6d8-8d73-4f393745e513', 'firstName': 'Ville', 'lastName': 'Reima'}, 'businessUnit': {'guid': 'f6d9f1e8-afae-1a74-5bbd-54d840a3e40e', 'name': 'Tietomallintaminen', 'companyGuid': '972b60e7-f511-9e96-5231-b066c16b3007', 'compa

2023-07-10 20:34:49.772 | SUCCESS  | src.logic.severa.base_client:get_with_retries:193 - HTTP/2 GET projects/7dbc27c9-7b75-5b5b-3fb4-8d67689c6a8b/projectforecasts in 0.11s.
2023-07-10 20:34:49.886 | SUCCESS  | src.logic.severa.base_client:get_with_retries:193 - HTTP/2 GET projects/7dbc27c9-7b75-5b5b-3fb4-8d67689c6a8b/projectforecasts in 0.11s.


len of forecasts:  4  /  1


In [20]:
billing

,id,internal_guid,start_date,end_date,project,value,billing,expense,revenue,labor_expense,user,forecast_date,_id
0,billing,2e1893e7-2a19-f04a-84eb-77f36d1e9b66,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,bbe91a59-1ae7-0230-0aea-090c8f31817e,100.0,100.0,0,0,0,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-07-10 00:00:00+00:00,ANqt4a1QgmPzcMs0wlZBRA==
1,billing,a7b6ba70-b289-c43f-e272-d1f0dde68585,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,b5432d39-bbb3-cd66-1e3f-c75411db4a95,1430.0,1430.0,0,0,0,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,2023-07-10 00:00:00+00:00,AKQjosKYW6ICtT5RIu1Bug==
2,billing,4ee052a5-b6f0-01b1-3123-5ccab1129ae6,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,0.0,0.0,0,0,624,78ddf0fe-9e39-5e5e-5096-a80409925de9,2023-07-10 00:00:00+00:00,AB+cvNqeiFyso0440tqwLA==
3,billing,43e25be8-5142-ba12-bcc3-7e6a00e8f55e,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,a3fa82b7-7692-4f84-14dc-4ee4e7a053a3,2900.0,2900.0,0,0,0,eb8d73da-8c1c-7996-b085-332e777dff1a,2023-07-10 00:00:00+00:00,ABfGkASomgmI713zoMnvyw==
4,billing,4e7d8e96-a31c-de9f-19b2-1a3fbe175d93,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,fc863117-ad5d-d5ca-fc48-a4262b82ef1a,2700.0,2700.0,0,0,0,78ddf0fe-9e39-5e5e-5096-a80409925de9,2023-07-10 00:00:00+00:00,APMbv5D99ysNOV7wh9UukQ==
5,billing,1193c5fc-3503-fc3e-c26d-14b639a30b40,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,41c84efd-9c66-ed83-2c50-7a638d5080f4,1170.0,1170.0,0,0,0,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-07-10 00:00:00+00:00,ADA04+96brsCUIukGAC+wA==
6,billing,4b5e04ed-2810-71a6-d488-23269c0439a5,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,c9ed2f7d-15cc-8f70-6456-319a8389ee8c,1000.0,1000.0,0,0,0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-10 00:00:00+00:00,ABFiIruNhV7T1vVtIkuApQ==
7,billing,cf4781b8-105c-f56a-9a17-6217b9801ddd,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,2363278a-275f-ce2c-7a3b-5beea945c845,1300.0,1300.0,0,0,0,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,2023-07-10 00:00:00+00:00,AARP01gh+ZcW9c2wg01gDQ==
8,billing,2371e7a5-472e-fa33-3ea6-304b3468e7b3,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,988c51fd-a7c9-1443-18b9-b9d9f51fb12e,1400.0,1400.0,0,0,0,af48336f-0979-b971-4927-9b09d53a3cef,2023-07-10 00:00:00+00:00,AObbAcAvwYn7SYU4J2cWPQ==
9,billing,850131ca-707c-6401-870d-bcec52e02e3e,2023-08-01 00:00:00+00:00,2023-08-31 23:59:59.999999+00:00,dece01c7-5324-c4b4-2bab-497009174f92,1300.0,1300.0,0,0,0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-07-10 00:00:00+00:00,APJq2pVsoYzAKffFIkTGxQ==


In [22]:
billing.pivot_table(
    "value",
    index=["user"],
    columns="id",
    aggfunc="sum",
    fill_value=0,
    dropna=False,
).reset_index()

# billing[billing.user == "2f365dcd-6ada-f6d8-8d73-4f393745e513"]

id,user,billing
0,0d100bc1-8376-7ac5-7bbb-055cdc20497d,7360.0
1,2441bfbf-78f2-0173-b0a1-8634ce167a0d,9470.0
2,2ed8156d-f3f9-2212-c739-b0d9f7ac81ca,2500.0
3,2f365dcd-6ada-f6d8-8d73-4f393745e513,12000.0
4,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,11308.0
5,78ddf0fe-9e39-5e5e-5096-a80409925de9,5500.0
6,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,13010.0
7,af48336f-0979-b971-4927-9b09d53a3cef,2700.0
8,eb8d73da-8c1c-7996-b085-332e777dff1a,17300.6


In [27]:
import asyncio
import src.logic.severa.rate_limiter

reload(src.logic.severa.rate_limiter)

async with src.logic.severa.rate_limiter.Limiter() as l:
    gets = [l.get(f"req {i}") for i in range(15)]
    await asyncio.gather(*gets)

CancelledError: 

In [11]:
base = src.database.database.Base("kpi-dev-02", "billing")
latest_date = base.find_max_value("forecast_date")

billing_f = base.find({"forecast_date": latest_date})

2023-07-10 16:07:15.981 | INFO     | src.database.database:find:90 - [billing] Query '{'forecast_date': datetime.datetime(2023, 7, 9, 0, 0)}' resulted in 68 results in 0.31s.


In [17]:
x = src.util.process.unravel(billing_f)
x[(x.date >= "2023-08-01") & (x.date < "2023-09-01")].pivot_table(
    "value",
    index=["user"],
    columns="id",
    aggfunc="sum",
    fill_value=0,
    dropna=False,
).reset_index()

id,user,billing
0,0d100bc1-8376-7ac5-7bbb-055cdc20497d,6110.0
1,2441bfbf-78f2-0173-b0a1-8634ce167a0d,9470.0
2,2ed8156d-f3f9-2212-c739-b0d9f7ac81ca,2000.0
3,2f365dcd-6ada-f6d8-8d73-4f393745e513,7000.0
4,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,6238.0
5,78ddf0fe-9e39-5e5e-5096-a80409925de9,5500.0
6,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,5700.0
7,af48336f-0979-b971-4927-9b09d53a3cef,1400.0
8,eb8d73da-8c1c-7996-b085-332e777dff1a,10090.6


In [15]:
u = src.severa.process.unravel(hours)
c = src.severa.process.cull_before(
    u,
    t.floor("day"),
    ["workhours", "saleswork"],  # absences?
    inclusive=False,
)
d = (
    c.groupby(["user", "id", "date", "productive", "project"], dropna=False)["value"]
    .sum()
    .reset_index()
)

d.id.unique()

C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(


<StringArray>
['absences', 'maximum', 'saleswork', 'workhours']
Length: 4, dtype: string

In [4]:
base = src.database.Base("kpi-dev-02", "billing")
billing_f = base.find({"forecast_date": arrow.utcnow().floor("day").datetime})

base = src.database.Base("kpi-dev-02", "hours")
hours_f = base.find({"forecast_date": arrow.utcnow().floor("day").datetime})

2023-07-04 20:49:38.562 | INFO     | src.database:find:90 - [billing] Query '{'forecast_date': datetime.datetime(2023, 7, 4, 0, 0, tzinfo=tzutc())}' resulted in 71 results in 1.69s.
2023-07-04 20:49:40.817 | INFO     | src.database:find:90 - [hours] Query '{'forecast_date': datetime.datetime(2023, 7, 4, 0, 0, tzinfo=tzutc())}' resulted in 226 results in 2.23s.


In [73]:
b = (
    src.severa.process.cull_before(
        src.severa.process.unravel(pd.concat([billing, billing_f], ignore_index=True)),
        arrow.utcnow().floor("day"),
        ["workhours", "billing"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)
b[b.value != 0]

cols = list(set(hours.columns) & set(hours_f.columns))
h1 = src.severa.process.sanitize_dates(
    hours, ["date", "start_date", "end_date", "forecast_date"]
)
h1 = src.severa.process.unravel(h1)

tot_hours1 = (
    y[y.id.isin(["workhours", "absences"])]
    .groupby(["user", "date"])["value"]
    .sum()
    .reset_index()
    .copy()
)
tot_hours1["id"] = "cost"
tot_hours1["value"] = cost.apply(lambda x: us2[x.user] * x.value, axis=1)


h2 = src.severa.process.sanitize_dates(
    hours_f, ["date", "start_date", "end_date", "forecast_date"]
)
h2 = src.severa.process.unravel(h2)

h3 = h1.merge(h2, on=cols, how="outer")

h = (
    src.severa.process.cull_before(
        h3,
        arrow.utcnow().floor("day"),
        ["workhours", "saleswork", "absences"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)

y = pd.concat([b[b.value != 0], h[h.value != 0]], ignore_index=True)
y = y[y.date.between(span.start.datetime, span.end.datetime)]

realized_cost = (
    y[(y.date <= arrow.utcnow().datetime) & (y.id.isin(["workhours", "absences"]))]
    .groupby(["user", "date"])["value"]
    .sum()
    .reset_index()
    .copy()
)


forecasted_cost = (
    y[(y.date > arrow.utcnow().datetime) & (y.id.isin(["maximum"]))]
    .groupby(["user", "date"])["value"]
    .sum()
    .reset_index()
    .copy()
)
print(forecasted_cost.value.mean())
print(forecasted_cost.sample(30))

cost = pd.concat([realized_cost, forecasted_cost], ignore_index=True)

cost["id"] = "cost"
cost["value"] = cost.apply(lambda x: us2[x.user] * x.value, axis=1)
cost.sample(50)

pd.concat([y, cost], ignore_index=True)

C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(
C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(
C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

7.162210338680927
                                      user                      date  value
59    0d100bc1-8376-7ac5-7bbb-055cdc20497d 2023-09-26 00:00:00+00:00    7.5
1059  2f365dcd-6ada-f6d8-8d73-4f393745e513 2024-09-25 00:00:00+00:00    7.5
1488  30c9d126-7cdc-53fc-3b05-7f98b25e3c98 2024-12-12 00:00:00+00:00    7.5
467   2441bfbf-78f2-0173-b0a1-8634ce167a0d 2023-11-13 00:00:00+00:00    7.5
3070  eb8d73da-8c1c-7996-b085-332e777dff1a 2023-10-23 00:00:00+00:00    7.5
1621  5cd93092-3385-3e89-003a-9fe6d03a424d 2024-01-02 00:00:00+00:00    1.5
1666  5cd93092-3385-3e89-003a-9fe6d03a424d 2024-03-05 00:00:00+00:00    1.5
511   2441bfbf-78f2-0173-b0a1-8634ce167a0d 2024-01-18 00:00:00+00:00    7.5
2214  78ddf0fe-9e39-5e5e-5096-a80409925de9 2024-11-11 00:00:00+00:00    7.5
1039  2f365dcd-6ada-f6d8-8d73-4f393745e513 2024-08-28 00:00:00+00:00    7.5
1023  2f365dcd-6ada-f6d8-8d73-4f393745e513 2024-08-06 00:00:00+00:00    7.5
325   0d100bc1-8376-7ac5-7bbb-055cdc20497d 2024-10-15 00:00:00+00:00  

,user,id,date,value
0,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-01 00:00:00+00:00,320.0
1,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-02 00:00:00+00:00,320.0
2,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-03 00:00:00+00:00,320.0
3,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-04 00:00:00+00:00,320.0
4,0d100bc1-8376-7ac5-7bbb-055cdc20497d,billing,2023-08-07 00:00:00+00:00,320.0
...,...,...,...,...
11036,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-17 00:00:00+00:00,363.75
11037,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-18 00:00:00+00:00,363.75
11038,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-19 00:00:00+00:00,363.75
11039,eb8d73da-8c1c-7996-b085-332e777dff1a,cost,2024-12-20 00:00:00+00:00,363.75


In [52]:
a = billing.groupby(["user", "id", "date"])["value"].sum().reset_index()
b = (
    src.severa.process.cull_before(
        src.severa.process.unravel(billing_f),
        arrow.utcnow().floor("day"),
        ["workhours", "billing"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)

c = pd.concat([a, b], ignore_index=True)
c  # 751
cc = c.groupby(["user", "id", "date"])["value"].sum().reset_index()

hours.groupby(["user", "id", "date"])["value"].sum().reset_index()
hours[pd.isna(hours.date)]

cols = list(set(hours.columns) & set(hours_f.columns))
h1 = src.severa.process.sanitize_dates(
    hours, ["date", "start_date", "end_date", "forecast_date"]
)
h2 = src.severa.process.sanitize_dates(
    hours_f, ["date", "start_date", "end_date", "forecast_date"]
)
h = h1.merge(h2, on=cols, how="outer")

x = (
    src.severa.process.cull_before(
        src.severa.process.unravel(h),
        arrow.utcnow().floor("day"),
        ["workhours", "saleswork", "abcenses", "billing"],
        inclusive=False,
    )
    .groupby(["user", "id", "date"])["value"]
    .sum()
    .reset_index()
)
print(len(x))
print(len(cc))

z = pd.concat([x, cc], ignore_index=True)
print(len(z))
print(len(z[z.value != 0]))
z[z.value != 0].sample(50)

C:\Users\vireima\tie-dashboard\src\severa\process.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(


33804
751
34555
23972


,user,id,date,value
17685,5cd93092-3385-3e89-003a-9fe6d03a424d,maximum,2024-04-11 00:00:00+00:00,1.5
32818,eb8d73da-8c1c-7996-b085-332e777dff1a,maximum,2026-01-02 00:00:00+00:00,7.5
16035,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,workhours,2024-09-20 00:00:00+00:00,1.015568
5616,2441bfbf-78f2-0173-b0a1-8634ce167a0d,maximum,2022-08-31 00:00:00+00:00,7.5
30767,eb8d73da-8c1c-7996-b085-332e777dff1a,maximum,2020-05-22 00:00:00+00:00,7.5
3740,0d100bc1-8376-7ac5-7bbb-055cdc20497d,workhours,2023-07-05 00:00:00+00:00,0.175562
6849,2441bfbf-78f2-0173-b0a1-8634ce167a0d,maximum,2026-01-15 00:00:00+00:00,7.5
13138,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,maximum,2024-06-06 00:00:00+00:00,7.5
7365,2441bfbf-78f2-0173-b0a1-8634ce167a0d,maximum,2027-06-15 00:00:00+00:00,7.5
20848,78ddf0fe-9e39-5e5e-5096-a80409925de9,maximum,2025-02-26 00:00:00+00:00,7.5


In [85]:
# print(hours.dtypes)

k = pd.DataFrame(
    data={
        "start_date": pd.NaT,
        "end_date": pd.NaT,
        "date": arrow.get("2023-01-01T12:00+0200").datetime,
        "value": [10.0, 8.0, 13.0],
    },
    index=[0, 1, 2],
)

cols = ["start_date", "end_date", "date"]
subset = k.loc[:, cols]
for c in cols:
    k[c] = pd.to_datetime(k.loc[:, c], utc=True)

k.date.dt.tz is datetime.timezone.utc

True

In [81]:
d = src.severa.process.unravel(hours)
d2 = src.severa.process.cull_before(d, arrow.utcnow().floor("day"), ["workhours"])
d3 = d2[("2023-01-01" <= d2.date) & (d2.date < "2023-10-01")]
# d3["total"] = d3.absences + d3.saleswork + d3.workhours
d3["total_cost"] = d3.value * d3.cost
# pd.pivot_table(d3, values=["value", "total_cost"], columns=["id"], index=[d3.date.dt.month, "name"], aggfunc=sum, fill_value=0).reset_index()

billing["name"] = billing.user.map(us)
b1 = src.severa.process.unravel(billing)
b1.groupby([b1.date.dt.month])["value"].sum().reset_index()

d4 = d3[d3.id != "maximum"]
d4.groupby([d4.date.dt.month])["total_cost"].sum().reset_index()

C:\Users\vireima\tie-dashboard\src\severa\process.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(
C:\Users\vireima\AppData\Local\Temp\ipykernel_20900\3774696409.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3["total_cost"] = d3.value * d3.cost
C:\Users\vireima\tie-dashboard\src\severa\process.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,date,total_cost
0,1,65255.955645
1,2,60758.58871
2,3,69948.439516
3,4,55455.536342
4,5,65170.208535
5,6,45745.082325
6,7,32600.293436
7,8,35226.940083
8,9,30624.146885


In [63]:
hours[hours.productive.isna()].id.unique()

<StringArray>
['maximum', 'absences']
Length: 2, dtype: string

In [27]:
us = {user.guid: user.firstName for user in await f.users()}
d = src.severa.process.unravel(
    hours
)  # [hours.id.isin(["workhours", "saleshours", "absences"])])
# d = src.severa.process.unravel(hours[hours.id.isin(["maximum"])], arrow.get("2023-03-01").datetime, arrow.get("2023-09-01").datetime)
d["name"] = d.user.map(us)
x = src.severa.process.cull_before(d, arrow.utcnow().floor("day"), ["workhours"])


d2 = x[("2023-06-01" <= x.date) & (x.date < "2023-10-01")]


pd.pivot_table(
    d2,
    values=["value"],
    columns=["id"],
    index=[d2.date.dt.month, "name"],
    aggfunc=sum,
    fill_value=0,
).reset_index()
# hours[hours.id == "maximum"]

C:\Users\vireima\tie-dashboard\src\severa\process.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "_num_workdays"] = data.loc[:, :].apply(


date       name     value                                 
id                      absences  maximum    saleswork   workhours
0          6    Johanna      52.5     157.5   1.991613   42.294511
1          6       Joni      15.0     157.5      7.875  110.348308
2          6       Lari      15.0     157.5        0.0    141.8125
3          6  Margarita       2.0     157.5    2.67052  143.217196
4          6      Miika       0.0     157.5        0.0   14.136094
5          6       Panu       0.0      31.5        0.0         0.0
6          6       Topi      15.0     157.5        0.0   97.393574
7          6      Ville       0.0     157.5        0.0   160.68209
8          6  Vladimirs       7.5     157.5        0.0  103.772486
9          7    Johanna     150.0     157.5   1.991613    3.294101
10         7       Joni       0.0     157.5      7.875   56.144925
11         7       Lari     150.0     157.5        0.0         3.5
12         7  Margarita      37.5     157.5    2.67052   38.310908
13         7      Miika       0.0     157.5        0.0   10.193483
14         7       Panu       0.0      31.5        0.0         0.0
15         7       Topi       0.0     157.5        0.0   47.579986
16         7      Ville      75.0     157.5        0.0   18.985294
17         7  Vladimirs       0.0     157.5        0.0   67.896899
18         8    Johanna       0.0     172.5    2.18129   75.764315
19         8       Joni       0.0     172.5     15.325   73.492061
20         8       Lari       0.0     172.5  13.176601  106.382353
21         8  Margarita       0.0     172.5   2.924855    54.09331
22         8      Miika       0.0     172.5        0.0   13.164291
23         8       Panu       0.0      34.5        0.0         0.0
24         8       Topi       0.0     172.5        0.0  132.726064
25         8      Ville       7.5     172.5        0.0  122.102941
26         8  Vladimirs       0.0     172.5        0.0    87.85971
27         9    Johanna       0.0     157.5   8.908843   68.926114
28         9       Joni       0.0     157.5  13.992391   56.144925
29         9       Lari       0.0     157.5   12.03081   107.49443
30         9  Margarita       0.0     157.5    2.67052   26.292365
31         9      Miika       0.0     157.5        0.0   12.193483
32         9       Panu       0.0      31.5        0.0         0.0
33         9       Topi       0.0     157.5        0.0  102.866091
34         9      Ville       7.5     157.5        0.0  111.416667
35         9  Vladimirs       0.0     157.5        0.0   89.319735

In [139]:
hours.loc[hours.date.isna() & (hours.id == "maximum"), "user"]

d = pd.DataFrame(
    {"date": pd.date_range("2023-05-01", "2023-05-30", freq="D", tz="utc")}
)
d["mask"] = d.date.map(Finland().is_working_day).map(int)
d

x = (
    hours.loc[hours.date.isna() & (hours.id == "maximum"), :]
    .drop("date", axis=1)
    .merge(d, how="cross")
)
x["value"] *= x["mask"]

In [157]:
x = src.severa.process.unravel_maximum(hours)
hours[hours.id == "maximum"]

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-15 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
1,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-16 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
2,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-17 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
3,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-18 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
4,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-19 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
5,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-20 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
6,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-21 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
7,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-22 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00
8,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,2020-03-23 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-15 00:00:00+00:00


In [178]:
def unravel_maximum(data: pd.DataFrame) -> pd.DataFrame:
    min_date = data.loc[:, ["date", "start_date"]].min().min()
    max_date = data.loc[:, ["date", "end_date"]].max().max()
    print(min_date)
    print(max_date)
    dt_index = pd.date_range(min_date, max_date, freq="D", tz="utc")
    dates = pd.DataFrame({"date": dt_index})

    dates["workday_mask"] = dates.date.map(Finland().is_working_day).map(int)
    unraveled_subset = (
        data.loc[data.id == "maximum", :].drop("date", axis=1).merge(dates, how="cross")
    )
    unraveled_subset["value"] *= unraveled_subset["workday_mask"]

    # data.merge(unraveled_subset, on=["id", "user"])
    # data.loc[data.date.isna() & (data.id == "maximum"), :] = masking.drop(
    #    "workday_mask", axis=1
    # )

    return unraveled_subset


hours[hours.id == "maximum"]
hours[hours.id != "maximum"]

pd.concat(
    [unravel_maximum(hours).drop("workday_mask", axis=1), hours[hours.id != "maximum"]],
    ignore_index=True,
)

2020-03-15 00:00:00+00:00
2027-11-30 00:00:00+00:00


,user,value,id,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date,date
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-15 00:00:00+00:00
1,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-16 00:00:00+00:00
2,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-17 00:00:00+00:00
3,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-18 00:00:00+00:00
4,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,2020-03-19 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25915,0d100bc1-8376-7ac5-7bbb-055cdc20497d,73.5,saleswork,<NA>,f62568a5-36ce-5dc3-490d-a3a24031dba1,<NA>,<NA>,<NA>,2022-11-03 00:00:00+00:00,2025-12-01 00:00:00+00:00,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,2023-06-16 00:00:00+00:00,NaT
25916,2441bfbf-78f2-0173-b0a1-8634ce167a0d,22.0,saleswork,<NA>,e98dd28a-27be-a646-3141-bef922976214,<NA>,<NA>,<NA>,2023-04-25 00:00:00+00:00,2023-12-30 00:00:00+00:00,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2023-06-16 00:00:00+00:00,NaT
25917,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,12.6,saleswork,<NA>,1913890d-2f2f-c164-3e71-e532a6c96ebe,<NA>,<NA>,<NA>,2023-08-01 00:00:00+00:00,2024-10-31 00:00:00+00:00,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-16 00:00:00+00:00,NaT
25918,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,1.6,saleswork,<NA>,1913890d-2f2f-c164-3e71-e532a6c96ebe,<NA>,<NA>,<NA>,2025-05-01 00:00:00+00:00,2025-06-30 00:00:00+00:00,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,2023-06-16 00:00:00+00:00,NaT


In [198]:
import random

dat = pd.DataFrame(
    {
        "id": list("AAABBB"),
        "value": [10, 8, 3, 7, 1, 1],
        "start_date": pd.Timestamp("2023-06-01"),
        "end_date": pd.Timestamp("2023-06-08"),
    }
)
dat.loc[1, "start_date"] = pd.Timestamp("2023-05-28")
dat.loc[2, "end_date"] = pd.Timestamp("2023-06-9")

dat["date"] = dat.apply(
    lambda x: pd.date_range(start=x["start_date"], end=x["end_date"]), axis=1
)
dat["_num_workdays"] = dat.apply(
    lambda x: Finland().get_working_days_delta(
        x["start_date"].date(), x["end_date"].date()
    ),
    axis=1,
)

dat = dat.explode("date").reset_index(drop=True)
dat["_is_workday"] = dat.date.map(Finland().is_working_day).map(int)
dat

hours[hours.date.isna()].sample(20)

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date
516,2f365dcd-6ada-f6d8-8d73-4f393745e513,4.0,workhours,NaT,<NA>,efd040a7-b946-7f32-cdc7-5254d4707403,<NA>,False,e4a802ac-54f4-d3f2-6b0c-229f94118e7d,2023-06-01 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
545,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,25.0,workhours,NaT,<NA>,f1a3d420-e970-74be-7668-ab3104c00551,<NA>,False,1915e2b0-c434-0322-9175-f9454a283e70,2023-06-26 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
563,2441bfbf-78f2-0173-b0a1-8634ce167a0d,67.5,workhours,NaT,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,<NA>,False,94b5fc97-f7cc-657a-a4db-b5803afd737f,2023-02-28 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
546,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,91.0,workhours,NaT,<NA>,2363278a-275f-ce2c-7a3b-5beea945c845,<NA>,False,67e159dc-1266-5c1c-51fa-228c95a1ea73,2022-10-01 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
538,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,8.0,workhours,NaT,<NA>,bf48e17f-7503-7ec5-87a8-25ad5cebd210,<NA>,False,079766cf-f3fd-80df-b9dd-e9bfc9a012e2,2023-06-01 00:00:00+00:00,2023-06-25 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
555,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,27.0,workhours,NaT,<NA>,7dff98a8-da80-1006-4bb1-2bda3507c7d4,<NA>,False,fb38fbce-53da-62a6-57e2-5ff874ff5950,2023-05-12 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
530,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,10.0,workhours,NaT,<NA>,bc7710f2-37a0-d14b-fcd4-a9e4fe8a95dd,<NA>,False,39aad51a-e04d-7880-daa1-fef750b382d2,2023-06-01 00:00:00+00:00,2023-06-25 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
490,eb8d73da-8c1c-7996-b085-332e777dff1a,2.0,workhours,NaT,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,<NA>,False,94b5fc97-f7cc-657a-a4db-b5803afd737f,2023-02-28 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
514,2f365dcd-6ada-f6d8-8d73-4f393745e513,31.0,workhours,NaT,<NA>,c1e8f641-cadc-06ac-fa6a-334e8afaf814,<NA>,True,2b8f4b28-c55c-2326-66fd-6ad56748c059,2023-03-02 00:00:00+00:00,2023-06-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00
565,2441bfbf-78f2-0173-b0a1-8634ce167a0d,44.0,workhours,NaT,<NA>,dece01c7-5324-c4b4-2bab-497009174f92,<NA>,False,ff9a747b-a7de-1d49-4cca-adb8ffa19180,2023-04-07 00:00:00+00:00,2023-07-16 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00


In [301]:
y = src.severa.process.unravel(hours)

C:\Users\vireima\tie-dashboard\src\severa\process.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"_num_workdays"] = data.loc[:,:].apply(


In [302]:
y

,user,value,id,date,activity_type,project,productive,is_internal,phase,start_date,end_date,sold_by,forecast_date,xxx
0,eb8d73da-8c1c-7996-b085-332e777dff1a,0.0,maximum,2020-03-15 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
1,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-16 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
2,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-17 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
3,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-18 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
4,eb8d73da-8c1c-7996-b085-332e777dff1a,14595.0,maximum,2020-03-19 00:00:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,2020-03-15 00:00:00+00:00,2027-11-30 00:00:00+00:00,<NA>,2023-06-16 00:00:00+00:00,14595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40840,2441bfbf-78f2-0173-b0a1-8634ce167a0d,2.5,workhours,2023-06-15 00:00:00+00:00,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,True,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40841,2441bfbf-78f2-0173-b0a1-8634ce167a0d,0.5,workhours,2023-06-15 00:00:00+00:00,<NA>,2d4fbce0-6524-15c2-1180-348bb49ef047,True,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40842,2441bfbf-78f2-0173-b0a1-8634ce167a0d,1.0,workhours,2023-06-15 00:00:00+00:00,<NA>,bad86762-4521-1d09-a992-473607c75c48,False,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
40843,2441bfbf-78f2-0173-b0a1-8634ce167a0d,1.0,workhours,2023-06-15 00:00:00+00:00,<NA>,bad86762-4521-1d09-a992-473607c75c48,False,<NA>,<NA>,NaT,NaT,<NA>,2023-06-16 00:00:00+00:00,<NA>
